# Basic Model Setup and attempt

In [1]:
import duckdb

## Sample formation

In [5]:
db = "../Data/fb.db"
conn = duckdb.connect(db)

In [169]:
basics = conn.sql("""
                    SELECT 
                    Season,
                    Div,
                    Date,
                    HomeTeam, 
                    AwayTeam, 
                    Win: (FTR = 'H') :: INT,
                    Draw: (FTR = 'D') :: INT,
                    Loss: (FTR = 'A') :: INT
                    FROM ResultsRaw 
                    WHERE Season IN ('1415', '1516', '1617', '1718')
                    """) 

In [170]:
Y = conn.sql("""
            SELECT *,
                Win: (FTR = 'W') :: INT,
                Draw: (FTR = 'D') :: INT,
                Loss: (FTR = 'L') :: INT

            FROM basics
            """
            )

BinderException: Binder Error: Referenced column "FTR" not found in FROM clause!
Candidate bindings: "HomeTeam"

In [ ]:
Y

In [171]:
stat_cols = conn.sql("DESCRIBE TeamStatsRecent").to_df()['column_name']

In [172]:
stat_cols

0              Div
1           Season
2             Team
3             Date
4       Pts_P5_Avg
5       G_F_P5_Avg
6      G_Ag_P5_Avg
7       S_F_P5_Avg
8      S_Ag_P5_Avg
9      ST_F_P5_Avg
10    ST_Ag_P5_Avg
11      F_F_P5_Avg
12     F_Ag_P5_Avg
13      C_F_P5_Avg
14     C_Ag_P5_Avg
15      Y_F_P5_Avg
16     Y_Ag_P5_Avg
17      R_F_P5_Avg
18     R_Ag_P5_Avg
Name: column_name, dtype: object

In [173]:
def homeAwayLabeler(cols, home=True, exclude=["Div", "Season", "Team", "Date"]):

    queryBldr = []
    if home:
        sfx = "Hm"
    else:
        sfx = "Aw"
    
    for col in stat_cols:
        if col in exclude:
            queryBldr.append("{} as {}".format(col, col))
        else:
            queryBldr.append("{} as {}_{}".format(col, col, sfx))

    return queryBldr

In [174]:
homeMap = homeAwayLabeler(list(stat_cols.values)) 

In [175]:
awayMap = homeAwayLabeler(list(stat_cols.values), home=False) 

In [176]:
query =  """
     SELECT 
         a.*,
         {}
         
     FROM basics a
     INNER JOIN TeamStatsRecent b
         ON a.HomeTeam = b.Team
         AND a.Date = b.Date
    """.format(",\n\t".join(homeMap[4:]))
           

In [177]:
print(query)


     SELECT 
         a.*,
         Pts_P5_Avg as Pts_P5_Avg_Hm,
	G_F_P5_Avg as G_F_P5_Avg_Hm,
	G_Ag_P5_Avg as G_Ag_P5_Avg_Hm,
	S_F_P5_Avg as S_F_P5_Avg_Hm,
	S_Ag_P5_Avg as S_Ag_P5_Avg_Hm,
	ST_F_P5_Avg as ST_F_P5_Avg_Hm,
	ST_Ag_P5_Avg as ST_Ag_P5_Avg_Hm,
	F_F_P5_Avg as F_F_P5_Avg_Hm,
	F_Ag_P5_Avg as F_Ag_P5_Avg_Hm,
	C_F_P5_Avg as C_F_P5_Avg_Hm,
	C_Ag_P5_Avg as C_Ag_P5_Avg_Hm,
	Y_F_P5_Avg as Y_F_P5_Avg_Hm,
	Y_Ag_P5_Avg as Y_Ag_P5_Avg_Hm,
	R_F_P5_Avg as R_F_P5_Avg_Hm,
	R_Ag_P5_Avg as R_Ag_P5_Avg_Hm
         
     FROM basics a
     INNER JOIN TeamStatsRecent b
         ON a.HomeTeam = b.Team
         AND a.Date = b.Date
    


In [178]:
homeJoin = conn.sql(query)

In [179]:
awayQuery =  """
     SELECT 
         a.*,
         {}
         
     FROM homeJoin a
     INNER JOIN TeamStatsRecent b
         ON a.AwayTeam = b.Team
         AND a.Date = b.Date
    """.format(",\n\t".join(awayMap[4:]))
        

In [180]:
print(awayQuery)


     SELECT 
         a.*,
         Pts_P5_Avg as Pts_P5_Avg_Aw,
	G_F_P5_Avg as G_F_P5_Avg_Aw,
	G_Ag_P5_Avg as G_Ag_P5_Avg_Aw,
	S_F_P5_Avg as S_F_P5_Avg_Aw,
	S_Ag_P5_Avg as S_Ag_P5_Avg_Aw,
	ST_F_P5_Avg as ST_F_P5_Avg_Aw,
	ST_Ag_P5_Avg as ST_Ag_P5_Avg_Aw,
	F_F_P5_Avg as F_F_P5_Avg_Aw,
	F_Ag_P5_Avg as F_Ag_P5_Avg_Aw,
	C_F_P5_Avg as C_F_P5_Avg_Aw,
	C_Ag_P5_Avg as C_Ag_P5_Avg_Aw,
	Y_F_P5_Avg as Y_F_P5_Avg_Aw,
	Y_Ag_P5_Avg as Y_Ag_P5_Avg_Aw,
	R_F_P5_Avg as R_F_P5_Avg_Aw,
	R_Ag_P5_Avg as R_Ag_P5_Avg_Aw
         
     FROM homeJoin a
     INNER JOIN TeamStatsRecent b
         ON a.AwayTeam = b.Team
         AND a.Date = b.Date
    


In [181]:
XY = conn.sql(awayQuery)

In [182]:
def selListQuery(table, cols):
    query = """
            SELECT
            {}
            FROM {}
            """.format(",".cols, table)

In [183]:
basics_cols = basics.to_df().columns
XY_cols = XY.to_df().columns

X_cols = [col for col in XY_cols if col not in basics_cols]

### Train test split

- Important to note that data points in Y won't be completely indendent of each other and X won't be completely independent of Y
- For now note it and revisit as this notebook is more about figuring out some of the basic infastructure and setting an initial benchmark
- Also when doing this more formally probably need to take more care about sampling evenly across season and division

In [184]:
conn.sql("SET THREADS = 1")

In [185]:
train = conn.sql("""
                SELECT *
                FROM XY
                USING SAMPLE 80 PERCENT (reservoir, 256);
                """
                )

In [186]:
test = conn.sql("""
              FROM XY
              ANTI JOIN train USING (Div, Season, Date, HomeTeam, AwayTeam);
              """
               )

## Scaling 

In [187]:
conn.sql("""
    CREATE OR REPLACE MACRO scaling_params(table_name, column_list) AS TABLE
    FROM query_table(table_name)
    SELECT
        avg(columns(column_list)) as 'avg_\\0',
        stddev_pop(columns(column_list)) as 'std_\\0',
        min(columns(column_list)) as 'min_\\0',
        max(columns(column_list)) as 'max_\\0',
        quantile_cont(columns(column_list), 0.25) AS 'q25_\\0',
        quantile_cont(columns(column_list), 0.50) AS 'q50_\\0',
        quantile_cont(columns(column_list), 0.75) AS 'q75_\\0',
        median(columns(column_list)) as 'median_\\0';    
    """)
conn.sql("""
    CREATE OR REPLACE MACRO standard_scaler(val, avg_val, std_val) AS
        (val - avg_val)/std_val;
    """)

In [188]:
scaleQueryBuilder = []
for col in X_cols:
     scaleQueryBuilder.append("{0}_SS: standard_scaler({0}, avg_{0}, std_{0})".format(col))
scaleQueryPart = ",\n".join(scaleQueryBuilder)

trainScaleQuery = """
    SELECT
    {0},
    {1}
    FROM train, 
        scaling_params('train', ['{2}'])
    """.format(",\n".join(basics_cols), scaleQueryPart, "',\n'".join(X_cols))

print(trainScaleQuery)


    SELECT
    Season,
Div,
Date,
HomeTeam,
AwayTeam,
Win,
Draw,
Loss,
    Pts_P5_Avg_Hm_SS: standard_scaler(Pts_P5_Avg_Hm, avg_Pts_P5_Avg_Hm, std_Pts_P5_Avg_Hm),
G_F_P5_Avg_Hm_SS: standard_scaler(G_F_P5_Avg_Hm, avg_G_F_P5_Avg_Hm, std_G_F_P5_Avg_Hm),
G_Ag_P5_Avg_Hm_SS: standard_scaler(G_Ag_P5_Avg_Hm, avg_G_Ag_P5_Avg_Hm, std_G_Ag_P5_Avg_Hm),
S_F_P5_Avg_Hm_SS: standard_scaler(S_F_P5_Avg_Hm, avg_S_F_P5_Avg_Hm, std_S_F_P5_Avg_Hm),
S_Ag_P5_Avg_Hm_SS: standard_scaler(S_Ag_P5_Avg_Hm, avg_S_Ag_P5_Avg_Hm, std_S_Ag_P5_Avg_Hm),
ST_F_P5_Avg_Hm_SS: standard_scaler(ST_F_P5_Avg_Hm, avg_ST_F_P5_Avg_Hm, std_ST_F_P5_Avg_Hm),
ST_Ag_P5_Avg_Hm_SS: standard_scaler(ST_Ag_P5_Avg_Hm, avg_ST_Ag_P5_Avg_Hm, std_ST_Ag_P5_Avg_Hm),
F_F_P5_Avg_Hm_SS: standard_scaler(F_F_P5_Avg_Hm, avg_F_F_P5_Avg_Hm, std_F_F_P5_Avg_Hm),
F_Ag_P5_Avg_Hm_SS: standard_scaler(F_Ag_P5_Avg_Hm, avg_F_Ag_P5_Avg_Hm, std_F_Ag_P5_Avg_Hm),
C_F_P5_Avg_Hm_SS: standard_scaler(C_F_P5_Avg_Hm, avg_C_F_P5_Avg_Hm, std_C_F_P5_Avg_Hm),
C_Ag_P5_Avg_Hm_SS: s

In [189]:
def scaleQueryBuilder(table, param_table, X_cols, Y_cols):
    scaleQueryBldr = []
    for col in X_cols:
         scaleQueryBldr.append("{0}_SS: standard_scaler({0}, avg_{0}, std_{0})".format(col))
    scaleQueryPart = ",\n\n\t\t".join(scaleQueryBldr)
    
    scaleQuery = """
        SELECT
            {0},
            {1}
        
        FROM {3}, 
            scaling_params('{4}', 
            ['{2}'])
        """.format(",\n\t\t".join(Y_cols), scaleQueryPart, "',\n\t\t'".join(X_cols), table, param_table)

    return scaleQuery
trainScaleQuery = scaleQueryBuilder("train", "train", X_cols, basics_cols)
testScaleQuery = scaleQueryBuilder("test", "train", X_cols, basics_cols)
print(trainScaleQuery)


        SELECT
            Season,
		Div,
		Date,
		HomeTeam,
		AwayTeam,
		Win,
		Draw,
		Loss,
            Pts_P5_Avg_Hm_SS: standard_scaler(Pts_P5_Avg_Hm, avg_Pts_P5_Avg_Hm, std_Pts_P5_Avg_Hm),

		G_F_P5_Avg_Hm_SS: standard_scaler(G_F_P5_Avg_Hm, avg_G_F_P5_Avg_Hm, std_G_F_P5_Avg_Hm),

		G_Ag_P5_Avg_Hm_SS: standard_scaler(G_Ag_P5_Avg_Hm, avg_G_Ag_P5_Avg_Hm, std_G_Ag_P5_Avg_Hm),

		S_F_P5_Avg_Hm_SS: standard_scaler(S_F_P5_Avg_Hm, avg_S_F_P5_Avg_Hm, std_S_F_P5_Avg_Hm),

		S_Ag_P5_Avg_Hm_SS: standard_scaler(S_Ag_P5_Avg_Hm, avg_S_Ag_P5_Avg_Hm, std_S_Ag_P5_Avg_Hm),

		ST_F_P5_Avg_Hm_SS: standard_scaler(ST_F_P5_Avg_Hm, avg_ST_F_P5_Avg_Hm, std_ST_F_P5_Avg_Hm),

		ST_Ag_P5_Avg_Hm_SS: standard_scaler(ST_Ag_P5_Avg_Hm, avg_ST_Ag_P5_Avg_Hm, std_ST_Ag_P5_Avg_Hm),

		F_F_P5_Avg_Hm_SS: standard_scaler(F_F_P5_Avg_Hm, avg_F_F_P5_Avg_Hm, std_F_F_P5_Avg_Hm),

		F_Ag_P5_Avg_Hm_SS: standard_scaler(F_Ag_P5_Avg_Hm, avg_F_Ag_P5_Avg_Hm, std_F_Ag_P5_Avg_Hm),

		C_F_P5_Avg_Hm_SS: standard_scaler(C_F_P5_Avg_Hm

In [190]:
trainSS = conn.sql(trainScaleQuery)

In [191]:
testSS = conn.sql(testScaleQuery)

## PCA

In [192]:
from sklearn.decomposition import PCA

- Drop null values due to lack of information at start of season for now

In [193]:
trainNn = conn.sql("""
        SELECT
        *
        FROM trainSS
        WHERE Pts_P5_Avg_Hm_SS IS NOT NULL
        """
        )

In [194]:
train_X = trainNn.to_df().loc[:,"Pts_P5_Avg_Hm_SS":]

In [195]:
pca = PCA(n_components=5)

In [196]:
pca.fit(train_X)

PCA(n_components=5)

In [197]:
train_X_pcs = pca.transform(train_X)

In [198]:
pca.explained_variance_ratio_

array([0.13128896, 0.12885087, 0.06289206, 0.05338598, 0.05053409])

- Somewhat arbitrary cut off at 5% variation explained for now

In [199]:
testNn = conn.sql("""
        SELECT
        *
        FROM testSS
        WHERE Pts_P5_Avg_Hm_SS IS NOT NULL
        """
        )

In [200]:
test_X = testNn.to_df().loc[:,"Pts_P5_Avg_Hm_SS":]

In [201]:
test_X_pcs = pca.transform(test_X)

## Logistic regression model
- Want something simple for now that produces a probability output and can is ok with low data

In [202]:
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier

ovr = OneVsRestClassifier(LogisticRegression(solver='liblinear'))

In [203]:
train_y = trainNn.to_df()[['Win', 'Draw', 'Loss']]

In [204]:
ovr.fit(X=train_X_pcs, y=train_y)

OneVsRestClassifier(estimator=LogisticRegression(solver='liblinear'))

In [219]:
test_y = testNn.to_df()[['Win', 'Draw', 'Loss']]
test_pred = ovr.predict(test_X_pcs)

In [226]:
test_pred.sum().sum()

401

In [236]:
(((test_y[test_pred.sum(axis = 1) > 0] - test_pred[test_pred.sum(axis = 1) > 0]) ** 2).sum(axis=1) == 0).sum()

199

In [237]:
199 / 401

0.49625935162094764

50% accuracy and only make guess with ~40% of data

In [251]:
test_pred_probs = ovr.predict_proba(test_X_pcs)

In [257]:
(test_pred_probs == test_pred_probs.max(axis = 1).reshape(-1,1)).sum(axis=0)

array([881,   0, 283])

- seems to be over predicting wins and vastly underpredicting draws

In [259]:
train_y.sum()

Win     2045
Draw    1200
Loss    1391
dtype: int64

- Some sample imbalance but some weirdness is going on

In [261]:
train_pred_probs = ovr.predict_proba(train_X_pcs)

In [262]:
(train_pred_probs == train_pred_probs.max(axis = 1).reshape(-1,1)).sum(axis=0)

array([3457,    0, 1179])

- Similar here

- Could be because technically there is a degree of order to the result

In [ ]:
cutoffs = [0.3333, 0.5, 0.7, 0.8]
for co in cutoffs:
    